# Data Loading

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
pd.set_option('display.float_format','{:.2f}'.format)

# Time Level 3 months (12-9)

## Time Level

In [ ]:
ctp = '24_jun_12'
ct = '24_jun_9'
tp1 = '25_jun_9'
tp2 = '25_jun_12'

## 3 months

In [ ]:
stypes = ['esales', 'nonesales']
atypes = ['act']
ks = [1]

In [ ]:
usag_df_tp1 = pd.read_parquet(f'Outputs/usag_{ct}_{tp1}_bucket_isocos_peinc_updated.parquet')
usag_df_tp2 = pd.read_parquet(f'Outputs/usag_{ctp}_{tp2}_bucket_isocos_peinc_updated.parquet')

In [ ]:
usag_df_tp2

In [ ]:
usag_df_1 = usag_df_tp1[['global_id', f'{ct}_Total_esales',f'{ct}_Total_nonesales',f'{tp1}_Total_esales', f'{tp1}_Total_nonesales',f'{ct}_{tp1}_esales_incremental_abs']]

In [ ]:
usag_df_2 = usag_df_tp2[['global_id','flag_gl_id',f'{tp2}_Total_esales',f'{ctp}_Total_esales',f'{ctp}_Total_nonesales',f'{tp2}_Total_nonesales',f'{ctp}_{tp2}_esales_incremental_abs',
                                  f'{ctp}_{tp2}_act_mon_esales_sd_1_segment',f'{ctp}_{tp2}_act_mon_nonesales_sd_1_segment',f'{ctp}_bucket']]

In [ ]:
usag_df = usag_df_1.merge(usag_df_2, how='inner',on='global_id')[['global_id','flag_gl_id',f'{ct}_Total_esales',f'{ct}_Total_nonesales',f'{ctp}_Total_esales',f'{ctp}_Total_nonesales',f'{tp1}_Total_esales', f'{tp1}_Total_nonesales',f'{ct}_{tp1}_esales_incremental_abs',
                                                                        f'{tp2}_Total_esales',f'{tp2}_Total_nonesales',f'{ctp}_{tp2}_esales_incremental_abs', f'{ctp}_{tp2}_act_mon_esales_sd_1_segment',f'{ctp}_{tp2}_act_mon_nonesales_sd_1_segment',f'{ctp}_bucket']]

In [ ]:
# usag_df = pd.read_csv(f'usag_diff_{tp1}_{tp2}_bucket_peinc_updated.csv')

In [ ]:
usag_df.columns

In [ ]:
usag_df[f'{tp2}_Total_esales'].sum()

In [ ]:
usag_df['diff_Total_esales_curr'] = usag_df[f'{tp2}_Total_esales'] - usag_df[f'{tp1}_Total_esales']
usag_df['diff_Total_nonesales_curr'] = usag_df[f'{tp2}_Total_nonesales'] - usag_df[f'{tp1}_Total_nonesales']

usag_df['diff_Total_esales_prev'] = usag_df[f'{ctp}_Total_esales'] - usag_df[f'{ct}_Total_esales']
usag_df['diff_Total_nonesales_prev'] = usag_df[f'{ctp}_Total_nonesales'] - usag_df[f'{ct}_Total_nonesales']


In [ ]:
usag_df['diff_esales_incremental_abs_updated'] = usag_df[f'{ctp}_{tp2}_esales_incremental_abs'] - usag_df[f'{ct}_{tp1}_esales_incremental_abs']

In [ ]:
usag_df['diff_esales_incremental_abs_updated'].sum()

In [ ]:
usag_df[f'{tp1}_{tp2}_act_mon_esales_sd_1_segment'] = usag_df[f'{ctp}_{tp2}_act_mon_esales_sd_1_segment']
usag_df[f'{tp1}_{tp2}_act_mon_nonesales_sd_1_segment'] = usag_df[f'{ctp}_{tp2}_act_mon_nonesales_sd_1_segment']

In [ ]:
core_segments = ['Stable','Growing','Declining']

In [ ]:
# with pd.ExcelWriter('Segments 25_q1.xlsx') as writer:
#     for seg1 in core_segments:
#         for seg2 in core_segments:
#             usag_df[(usag_df['24_jun_12_25_jun_12_act_mon_esales_sd_1_segment']==seg1) & (usag_df['24_jun_12_25_jun_12_act_mon_nonesales_sd_1_segment']==seg2)].to_excel(writer,sheet_name=f'{seg1}-{seg2}')



In [ ]:
segments = ['Unallocated', 'New', 'Growing', 'Stable', 'Declining', 'Not in Channel']
segments

In [ ]:
def crosstab_func(usag_df,atype,k,tp1,tp2):
    metric_rows = ['# of Customers', 'eSales_Inc', f'eSales prev', f'eSales curr']
    metric_cols = ['.','..', f'non-eSales prev', f'non-eSales curr']


    row_index = pd.MultiIndex.from_product([segments, metric_rows])
    col_index = pd.MultiIndex.from_product([segments, metric_cols])


    cross_tab = pd.DataFrame(index=row_index, columns=col_index)


    total_nonesales_prev = usag_df[f'diff_Total_nonesales_prev'].sum()
    total_nonesales_curr = usag_df[f'diff_Total_nonesales_curr'].sum()
    total_esales_prev = usag_df[f'diff_Total_esales_prev'].sum()
    total_esales_curr = usag_df[f'diff_Total_esales_curr'].sum()

    for e_seg in segments:
        for n_seg in segments:
            subset = usag_df[
                (usag_df[f'{tp1}_{tp2}_{atype}_mon_esales_sd_{str(k)}_segment'] == e_seg) &
                (usag_df[f'{tp1}_{tp2}_{atype}_mon_nonesales_sd_{str(k)}_segment'] == n_seg)
            ]

            num_customers = subset.shape[0]
            esales_prev = subset[f'diff_Total_esales_prev'].sum()
            esales_curr = subset[f'diff_Total_esales_curr'].sum()
            nonesales_prev = subset[f'diff_Total_nonesales_prev'].sum()
            nonesales_curr = subset[f'diff_Total_nonesales_curr'].sum()

            esales_inc = subset[f'diff_esales_incremental_abs_updated'].sum()

            cross_tab.loc[(e_seg, '# of Customers'), (n_seg, '.')] = num_customers
            cross_tab.loc[(e_seg, '# of Customers'), (n_seg, f'non-eSales prev')] = f"{nonesales_prev:.0f}"
            cross_tab.loc[(e_seg, '# of Customers'), (n_seg, f'non-eSales curr')] = f"{nonesales_curr:.0f}"
            cross_tab.loc[(e_seg, 'eSales_Inc'), (n_seg, f'non-eSales prev')] = f"{(nonesales_prev/total_nonesales_prev):.2%}"
            cross_tab.loc[(e_seg, 'eSales_Inc'), (n_seg, f'non-eSales curr')] = f"{(nonesales_curr/total_nonesales_curr):.2%}"

            cross_tab.loc[(e_seg, 'eSales_Inc'), (n_seg, '.')] = f"{esales_inc:.0f}"

            cross_tab.loc[(e_seg, f'eSales prev'), (n_seg, '.')] = f"{esales_prev:.0f}"
            cross_tab.loc[(e_seg, f'eSales curr'), (n_seg, '.')] = f"{esales_curr:.0f}"
            cross_tab.loc[(e_seg, f'eSales prev'), (n_seg, '..')] = f"{(esales_prev/total_esales_prev):.2%}"
            cross_tab.loc[(e_seg, f'eSales curr'), (n_seg, '..')] = f"{(esales_curr/total_esales_curr):.2%}"


    return cross_tab

In [ ]:
sheet_dict = {}
for atype in atypes:
    for k in ks:
        sheet_name = f"cross_tab_{tp1}_{tp2}_{atype}_mon_sd_{str(k)}"
        sheet_dict[sheet_name] = crosstab_func(usag_df,atype, k,tp1, tp2)
print("done")

In [ ]:
with pd.ExcelWriter(f"v2_crosstab_output_diff_bucket_isocos_pe_{tp1}_{tp2}.xlsx") as writer:
    for sheet_name, df in sheet_dict.items():
        df.to_excel(writer,sheet_name = sheet_name[:31])

In [ ]:
usag_df['diff_esales_incremental_abs_updated'].sum()

In [ ]:
usag_df.to_parquet(f'Outputs/usag_diff_{tp1}_{tp2}_bucket_isocos_peinc_updated.parquet')

In [ ]:
usag_df.groupby(f'{ctp}_bucket')['diff_esales_incremental_abs_updated'].sum()